In [7]:
# Building LDA model

import gensim
import pandas as pd
import csv
import ast
from gensim import corpora, models

n_topics = 4

docs = []
with open('tester.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        docs.append((ast.literal_eval(row[1])))
        
        
# TF-IDF CORPUS (CHANGE VAR NAMES)
tfidf = models.TfidfModel(corpus)
corpus= tfidf[bow_corpus]

dictionary = gensim.corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]
#=====LDA using word of bags====
lda_model = gensim.models.LdaMulticore(corpus, num_topics=n_topics, id2word=dictionary, passes=10)
#lda_model.save('model.gensim')

#For each topic, we will explore the words occuring in that topic and its relative weight.
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

/Users/amp/opt/anaconda3/envs/lda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic: 0 
Words: 0.019*"name" + 0.017*"differ" + 0.016*"use" + 0.016*"articl" + 0.015*"correct" + 0.014*"identif" + 0.013*"link" + 0.010*"object" + 0.010*"wrong" + 0.010*"also"
Topic: 1 
Words: 0.029*"item" + 0.020*"merg" + 0.018*"categori" + 0.013*"page" + 0.011*"name" + 0.010*"ad" + 0.009*"remov" + 0.009*"link" + 0.008*"conflict" + 0.008*"statu"
Topic: 2 
Words: 0.036*"item" + 0.015*"style" + 0.014*"remov" + 0.012*"color" + 0.011*"see" + 0.010*"page" + 0.010*"person" + 0.010*"special" + 0.009*"name" + 0.009*"sup"
Topic: 3 
Words: 0.026*"link" + 0.017*"item" + 0.016*"one" + 0.013*"page" + 0.011*"date" + 0.010*"two" + 0.010*"articl" + 0.009*"add" + 0.008*"would" + 0.007*"differ"


In [8]:
# Visualizing with tSNE

from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
import numpy as np
import matplotlib.colors as mcolors

# Get topic weights
topic_weights = []
for i, row_list in enumerate(lda_model[corpus]):
    topic_weights.append([w for i, w in row_list])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

/Users/amp/opt/anaconda3/envs/lda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 7114 samples in 0.002s...
[t-SNE] Computed neighbors for 7114 samples in 0.111s...
[t-SNE] Computed conditional probabilities for sample 1000 / 7114
[t-SNE] Computed conditional probabilities for sample 2000 / 7114
[t-SNE] Computed conditional probabilities for sample 3000 / 7114
[t-SNE] Computed conditional probabilities for sample 4000 / 7114
[t-SNE] Computed conditional probabilities for sample 5000 / 7114
[t-SNE] Computed conditional probabilities for sample 6000 / 7114
[t-SNE] Computed conditional probabilities for sample 7000 / 7114
[t-SNE] Computed conditional probabilities for sample 7114 / 7114
[t-SNE] Mean sigma: 0.000046
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.784081
[t-SNE] KL divergence after 1000 iterations: 0.500603


Loading BokehJS ...

In [9]:
# Using pyLDAvis

import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

/Users/amp/opt/anaconda3/envs/lda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.026536  0.008142       1        1  33.022652
0      0.110982  0.025538       2        1  24.265520
2     -0.040976 -0.113411       3        1  23.278604
1     -0.096541  0.079732       4        1  19.433223, topic_info=         Term         Freq        Total Category  logprob  loglift
370   identif   293.000000   293.000000  Default  30.0000  30.0000
7        item  1740.000000  1740.000000  Default  29.0000  29.0000
51   categori   340.000000   340.000000  Default  28.0000  28.0000
247      merg   528.000000   528.000000  Default  27.0000  27.0000
318     style   331.000000   331.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
38        one    97.108416   774.728465   Topic4  -5.1228  -0.4385
293      list    69.701759   314.666364   Topic4  -5.4544   0.1309
174    differ    84.257984   701.593148   Topic4  -5.2647  -0.4813
40      delet    63.724433   229.849344   Topic4  -5.5441   0.3553
2      articl    68.319874   780.189650   Topic4  -5.4744  -0.7972

[319 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
5276      3  0.947479    abu
1044      2  0.934448   acid
977       1  0.844076  actor
977       2  0.043286  actor
977       3  0.086572  actor
...     ...       ...    ...
28        2  0.537176  wrong
28        3  0.122705  wrong
28        4  0.040902  wrong
1374      1  0.149786   yair
1374      4  0.848785   yair

[621 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])